<a href="https://colab.research.google.com/github/Tstrebe2/predicting-text-difficulty/blob/tim-updates/code/tim-text-cleaning-pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

!{sys.executable} -m pip install pyspark==3.1.2 -q
!{sys.executable} -m pip install spark-nlp==4.2.0 -q
# !{sys.executable} -m pip install -U spacy==3.4.1 -q

# !{sys.executable} -m spacy download en_core_web_lg -q

     |████████████████████████████████| 212.4 MB 70 kB/s 
     |████████████████████████████████| 198 kB 9.4 MB/s 
     |████████████████████████████████| 641 kB 7.9 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

!wget https://raw.githubusercontent.com/Tstrebe2/predicting-text-difficulty/main/assets/WikiLarge_Train.csv -q
!wget https://raw.githubusercontent.com/mahavivo/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt -q
!wget https://raw.githubusercontent.com/Tstrebe2/predicting-text-difficulty/main/assets/dale_chall.txt -q
!wget https://raw.githubusercontent.com/Tstrebe2/predicting-text-difficulty/main/assets/AoA_51715_words.csv -q
!wget https://raw.githubusercontent.com/Tstrebe2/predicting-text-difficulty/main/assets/Concreteness_ratings_Brysbaert_et_al_BRM.txt -q

Mounted at /content/drive


In [3]:
import pandas as pd

aoa = pd.read_csv('/content/AoA_51715_words.csv', 
                     encoding_errors='ignore', 
                     usecols=['Lemma_highest_PoS', 'AoA_Kup_lem'],
                     ).rename({'Lemma_highest_PoS':'lemma', 'AoA_Kup_lem':'aoa'}, axis=1)

aoa = aoa.groupby('lemma').first().to_dict()['aoa']

conc = (pd.read_csv('/content/Concreteness_ratings_Brysbaert_et_al_BRM.txt', 
                   sep='\t',
                   usecols=['Word', 'Bigram', 'Conc.M'])
        .rename({'Word':'word', 'Bigram':'bigram', 'Conc.M':'conc_mean'}, axis=1))

def split_word(x):
  if x['bigram'] == 0:
    word_or_phrase = x['word']
  else:
    word_or_phrase = tuple(x['word'].split(' '))

  return {'word':word_or_phrase, 'conc_mean':x['conc_mean'] }

conc = conc.apply(split_word, axis=1, result_type='expand').set_index('word').to_dict()['conc_mean']

d_chall = set(pd.read_csv('/content/dale_chall.txt', names=['word'])['word'].tolist())

In [51]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.ml import Pipeline
import pyspark.sql.functions as F
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import SentenceDetector, Tokenizer, Normalizer, Lemmatizer
import sparknlp
from pyspark.sql.types import StringType, ArrayType, FloatType, StructType
import numpy as np
import string

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.2.0")\
        .getOrCreate()

df = spark.read.csv('/content/WikiLarge_Train.csv', header=True)
df.createOrReplaceTempView('wiki')

regex1, replace1 = r'-LRB-', '('
regex2, replace2 = r'-RRB-', ')'
regex3, replace3 = r"\'\' *", ''
regex4, replace4 = r", ; *", ''
regex5, replace5 = r"; , *", ''
regex6, replace6 = r", , *", ''
regex7, replace7 = r"; ; *", ''
regex8, replace8 = r"[(] [;,-]* [)] ", ''
regex9, replace9 = r"[(] +[)] *", ''
regex10, replace10 = r" km ", "kilometers"
regex11, replace11 = r"[0-9]+(km) ", "kilometers"
regex12, replace12 = r" mph "," miles per hour "
regex13, replace13 = r"° C ","degrees Celsius"
regex14, replace14 = r"° F ","degrees Farenheit"
regex15, replace15 = r"°","degrees"
regex16, replace16 = r" %"," percent"
regex17, replace17 = r" cm"," centimeters"
regex18, replace18 = r" kg "," kilograms "

iterable = ((regex1, replace1), (regex2, replace2), (regex3, replace3),
            (regex4, replace4), (regex5, replace5), (regex6, replace6),
            (regex7, replace7), (regex8, replace8), (regex9, replace9),
            (regex10,replace10), (regex11,replace11),(regex12, replace12),
            (regex13, replace13),(regex14,replace14),(regex15,replace15),
            (regex16,replace16), (regex17,replace17),(regex18,replace18))

for regex, replace in iterable:
  query = f"""
  SELECT
    regexp_replace(original_text, '{regex}', '{replace}') as original_text, 
    label 
  FROM wiki;"""
  df = spark.sql(query)
  df.createOrReplaceTempView('wiki')

query = r"SELECT * FROM wiki WHERE LENGTH(original_text) > 20;"
df = spark.sql(query)
df.createOrReplaceTempView('wiki')

documentAssembler = DocumentAssembler()\
    .setInputCol("original_text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

lemmatizer = Lemmatizer() \
    .setInputCols(["token"]) \
    .setOutputCol("lemma") \
    .setDictionary("./AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")

nlp_pipeline = Pipeline(stages=[documentAssembler,
                               tokenizer,
                               lemmatizer,
                               ])

nlp_pipeline = nlp_pipeline.fit(df)
df = nlp_pipeline.transform(df)
df.createOrReplaceTempView('wiki')

def get_d_chall(x):
  easy_count = 0
  word_count = 0

  for token in x:
    if token in string.punctuation:
      continue

    token_lower = token.lower()

    if token_lower in d_chall:
      easy_count += 1

    word_count += 1

  difficult_count = (word_count-easy_count)

  return (0.1579 * ((difficult_count/word_count) * 100) + 0.0496 * word_count
          if word_count else 0.0)


def get_aoa(x):
  arr = [aoa[w.lower()] for w in x if w.lower() in aoa]
  if len(arr) > 0:
    return arr
  else:
    return [0.0] 

def get_conc_rating(x):
  ret_val = []

  bigrams = [(f.lower(), s.lower()) for f, s in zip(x[:-1], x[1:])]
  cont = False
  
  for bigram in bigrams:
    if cont:
      cont = False
      continue

    if bigram in conc:
      cont = True
      ret_val.append(conc[bigram])
    elif bigram[0] in conc:
      ret_val.append(conc[bigram[0]])

  return ret_val

def get_num_lemmas(x):
  num_lemmas = 0.0

  for token in x:
    if token not in string.punctuation:
      num_lemmas += 1.0

  return num_lemmas
    
spark.udf.register('get_d_chall', get_d_chall, FloatType())
spark.udf.register('get_aoa', get_aoa, ArrayType(FloatType()))
spark.udf.register('get_conc_rating', get_conc_rating, ArrayType(FloatType()))
spark.udf.register('get_num_lemmas', get_num_lemmas, FloatType())
spark.udf.register('get_joined_text', lambda x: ' '.join(x), StringType())
spark.udf.register('array_mean', lambda x: float(np.mean(x)), FloatType())

query = r"""
SELECT 
  original_text, 
  get_joined_text(lemma.result) as lemmatized_text, 
  get_d_chall(lemma.result) as d_chall_score,
  get_aoa(lemma.result) as aoa,
  get_conc_rating(lemma.result) as conc_rating,
  get_num_lemmas(lemma.result) as num_lemmas,
  label
FROM wiki;
"""
df = spark.sql(query)
df.createOrReplaceTempView('wiki')

spark.udf.register('array_mean', lambda x: float(np.mean(x)), FloatType())

query = r"""
SELECT 
original_text, 
lemmatized_text,
d_chall_score,
array_mean(aoa) as aoa_mean, 
array_min(aoa) as aoa_min, 
array_max(aoa) as aoa_max, 
array_mean(conc_rating) as conc_rating_mean, 
array_min(conc_rating) as conc_rating_min, 
array_max(conc_rating) as conc_rating_max,
num_lemmas,
label
FROM wiki; 
"""
df = spark.sql(query)
df.createOrReplaceTempView('wiki')
df.show(5, 0)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+---------+-------+-------+----------------+---------------+---------------+----------+-----+
|original_text                                                                                                                                                                                                                                         |lemmatized_text                                                                                                                                                             

In [46]:
df = df.toPandas()

In [50]:
pd.set_option("display.max_rows", 80)

df.iloc[:, 2:].groupby('label').describe().T

label                               0              1
d_chall_score    count  190114.000000  205055.000000
                 mean        7.197394       7.891544
                 std         2.850988       2.472041
                 min         0.148800       0.000000
                 25%         5.278911       6.304934
                 50%         7.033356       7.808743
                 75%         8.787800       9.252926
                 max        18.716400      18.716400
aoa_mean         count  190111.000000  205050.000000
                 mean        5.627690       5.872704
                 std         1.065768       0.926203
                 min         0.000000       0.000000
                 25%         5.082880       5.373000
                 50%         5.522143       5.819523
                 75%         6.084188       6.289688
                 max        18.330000      17.620001
aoa_min          count  190114.000000  205055.000000
                 mean        3.440302       3.353729
                 std         0.889125       0.752399
                 min         0.000000       0.000000
                 25%         2.890000       2.890000
                 50%         3.440000       3.230000
                 75%         3.690000       3.690000
                 max        18.330000      17.620001
aoa_max          count  190111.000000  205050.000000
                 mean        9.897050      10.907201
                 std         2.665635       2.427625
                 min         0.000000       0.000000
                 25%         8.170000       9.330000
                 50%         9.840000      10.940000
                 75%        11.780000      12.500000
                 max        21.000000      25.000000
conc_rating_mean count  188262.000000  203963.000000
                 mean        2.691317       2.657542
                 std         0.378329       0.332931
                 min         1.190000       1.190000
                 25%         2.451667       2.443750
                 50%         2.656364       2.623333
                 75%         2.892846       2.836000
                 max         5.000000       5.000000
conc_rating_min  count  188262.000000  203963.000000
                 mean        1.496481       1.450270
                 std         0.357335       0.282830
                 min         1.040000       1.040000
                 25%         1.430000       1.420000
                 50%         1.430000       1.430000
                 75%         1.460000       1.430000
                 max         5.000000       5.000000
conc_rating_max  count  188262.000000  203963.000000
                 mean        4.430801       4.487875
                 std         0.595101       0.540891
                 min         1.190000       1.190000
                 25%         4.140000       4.230000
                 50%         4.680000       4.710000
                 75%         4.880000       4.880000
                 max         5.000000       5.000000
num_lemmas       count  190114.000000  205055.000000
                 mean       17.344545      22.004589
                 std         9.450984      10.996905
                 min         1.000000       0.000000
                 25%        11.000000      14.000000
                 50%        16.000000      20.000000
                 75%        22.000000      28.000000
                 max        76.000000      76.000000